# Une bibliothèque

Au cours de cet exercice, vous apprendrez à mettre sur pied une véritable base de données relationnelle. Cette base de données sera composée de trois tables reliées entre elles par un mécanisme de clé primaire et clé étrangère.

## Connexion à la base de données

Avant tout, importez le module python qui permet d’utiliser les instructions SQLite.

In [ ]:
# Your code here
import sqlite3

Mettez en place une connexion à un fichier `library.db`.

In [ ]:
# Your code here
connection = sqlite3.connect('../data/library.db')

Instanciez à présent sur cette connexion un objet `Cursor`.

In [ ]:
# Your code here
c = connection.cursor()

## La structure de la base de données

Prenez tout d’abord connaissance du fichier [*library.xml*](../data/library.xml). Il représente deux tables :
- *books*
- *authors*

La table *books* est composée des champs évidents comme *title*, *pub_year*, *summary* et *grade*, auxquels il faut rajouter ceux qui forment les attributs *ref_author*, *xml:lang*, *original_title*, *original_title_lang*.

Pour la table *authors*, on peut lister : *id_author*, *lastname*, *firstname*, *birth*, *death* et enfin *place*, qui est commun aux deux précédents.

### Créer la table *authors*

Avant de lancer une instruction `CREATE TABLE`, définissez pour chaque champ le type de données qu’il va enregistrer, décidez si vous souhaitez les rendre obligatoires ou non et attribuez-leur une éventuelle valeur par défaut. Enfin, décidez si l’un ou l’autre de ces champs devrait être considéré comme une clé primaire ou une clé étrangère.

|field|datatype|NULL|default|key|
|:-|:-:|:-:|:-:|:-:|
|id_author|TEXT|NOT NULL|-|PRIMARY KEY|
|lastname|TEXT|NOT NULL|-|-|
|firstname|TEXT|NOT NULL|-|-|
|birth|TEXT|NOT NULL|-|-|
|death|TEXT|NOT NULL|-|-|
|place|TEXT|NOT NULL|-|-|
|place|TEXT|NOT NULL|-|-|

Remarquez les champs `place` et `place` qui non seulement sont nommés de la même manière mais ont pour vocation à recevoir tous deux le même genre de données, à savoir le nom d’une localité. Mettons-les provisoirement de côté.

Ensuite, sur la base de cette liste, rédigez l’instruction SQL équivalente.

In [ ]:
# Your code here
sql = """CREATE TABLE IF NOT EXISTS authors (
id_author TEXT PRIMARY KEY,
lastname TEXT NOT NULL,
firstname TEXT NOT NULL,
birth TEXT NOT NULL,
death TEXT NOT NULL
)"""

Et exécutez-la sur l’objet `Cursor` que vous avez instancié plus tôt.

In [ ]:
# Your code here
c.execute(sql)

### Créer la table *books*

Comme pour la table *authors*, établissez une liste des champs de données avec les options éventuelles. Rajoutez un champ *id_book* afin de fournir chaque enregistrement d’un identifiant.

|field|datatype|NULL|default|key|
|:-|:-:|:-:|:-:|:-:|
|id_book|INTEGER|NOT NULL|-|PRIMARY KEY|
|ref_author|TEXT|NOT NULL|-|FOREIGN KEY|
|lang|TEXT|NOT NULL|-|-|
|title|TEXT|NOT NULL|-|-|
|original_title|TEXT|-|-|-|
|original_title_lang|TEXT|-|-|-|
|pub_year|INTEGER|NOT NULL|-|-|
|summary|TEXT|NOT NULL|-|-|
|grade|INTEGER|NOT NULL|-|-|

Traduisez ensuite cette liste sous forme d’instruction SQL.

In [ ]:
# Your code here
sql = """CREATE TABLE IF NOT EXISTS books (
id_book INTEGER PRIMARY KEY,
ref_author TEXT,
lang TEXT NOT NULL,
title TEXT NOT NULL,
original_title TEXT,
original_title_lang TEXT,
pub_year INTEGER NOT NULL,
summary TEXT NOT NULL,
grade TEXT NOT NULL,
FOREIGN KEY(ref_author) REFERENCES authors(id_author)
)"""

Exécutez maintenant l’instruction SQL.

In [ ]:
c.execute(sql)

### Créer la table *places*

Intéressons-nous maintenant aux champs *places* que nous avons mis de côté. Plutôt que d’insérer deux champs identiques dans la table *authors*, l’idée est de créer une troisième table qui identifierait un lieu de manière unique, référencée ensuite dans la table des auteurs. Définissez dans un premier temps cette table *places*.

|field|datatype|NULL|default|key|
|:-|:-:|:-:|:-:|:-:|
|id_place|INTEGER|NOT NULL|-|PRIMARY KEY|
|place|TEXT|NOT NULL|-|-|

Traduisez cette liste en une instruction SQL et exécutez-la.

In [ ]:
# Your code here
sql = """CREATE TABLE IF NOT EXISTS places (
id_place INTEGER PRIMARY KEY,
place TEXT NOT NULL
)"""
c.execute(sql)

### Mettre à jour la table *authors*

Prévoir pour la table *authors* deux champs de référence à la clé `id_place` de la table *places*. Réécrivez la liste des champs en intégrant cette contrainte.

|field|datatype|NULL|default|key|
|:-|:-:|:-:|:-:|:-:|
|id_author|TEXT|NOT NULL|-|PRIMARY KEY|
|ref_birthplace|INTEGER|NOT NULL|-|FOREIGN KEY|
|ref_deathplace|INTEGER|NOT NULL|-|FOREIGN KEY|
|lastname|TEXT|NOT NULL|-|-|
|firstname|TEXT|NOT NULL|-|-|
|birth|TEXT|NOT NULL|-|-|
|death|TEXT|NOT NULL|-|-|

Si la norme SQL permet de rajouter une clé étrangère à une table existante, la version allégée SQLite impose de redéfinir la table *authors*. Il vous faut par conséquent la supprimer avant !

In [ ]:
# Your code here
c.execute("DROP TABLE authors")

Avant de la redéfinir avec les deux champs de référence à `places.id_place`.

In [ ]:
# Your code here
sql = """CREATE TABLE IF NOT EXISTS authors (
id_author TEXT PRIMARY KEY,
ref_birthplace INTEGER,
ref_deathplace INTEGER,
lastname TEXT NOT NULL,
firstname TEXT NOT NULL,
birth TEXT NOT NULL,
death TEXT NOT NULL,
FOREIGN KEY(ref_birthplace) REFERENCES places(id_place),
FOREIGN KEY(ref_deathplace) REFERENCES places(id_place)
)"""
c.execute(sql)

### Constituer des index

Identifiez au sein des trois tables, les champs qui pourraient servir d’index.

|table|field|unique|
|:-|:-:|:-:|
|authors|lastname|-|
|books|title|-|
|books|original_title|-|
|places|place|UNIQUE|

Exécutez les instructions sql équivalentes.

In [ ]:
# Your code here
c.execute("CREATE INDEX IF NOT EXISTS lastname_index ON authors (lastname)")
c.execute("CREATE INDEX IF NOT EXISTS title_index ON books (title)")
c.execute("CREATE INDEX IF NOT EXISTS vo_title_index ON books (original_title)")
c.execute("CREATE INDEX IF NOT EXISTS place_index ON places (place)")

## Intégrer les données

À présent que la structure de la base de données est bien définie, il reste à enregistrer les données à l’intérieur.

### Lire le fichier XML

Cette étape suppose d’extraire les informations du fichier XML. Dans un premier temps, appelez le module Python qui propose une interface XML et chargez la racine du fichier dans une variable `root`.

In [ ]:
# Your code here
import xml.etree.ElementTree as ET

tree = ET.parse('../data/library.xml')
root = tree.getroot()

### Remplir la table *places*

Récupérez ensuite une liste dédoublonnée des lieux de vie et de mort renseignés pour les auteurs à l’intérieur des attributs `places`.

In [ ]:
# Your code here
places = set()

for author in root.iter('author'):
    birthplace = author.find('birth').get('place')
    deathplace = author.find('death').get('place')
    places.update({birthplace, deathplace})

À partir de cette liste, construisez une instruction SQL pour remplir la table *places*.

In [ ]:
# Your code here
sql = "INSERT INTO places VALUES (?, ?)"
c.executemany(sql, enumerate(places))

Si les transactions sont couronnées de succès, l’instruction suivante devrait renvoyer la liste des enregistrements :

In [ ]:
# Print all the records in the table 'places'
for row in c.execute("SELECT * FROM places"):
    print(row)

### Remplir la table *authors*

Répétez les opérations pour la table *authors*. Pour renseigner les clés étrangères `birthplace` et `deathplace`, vous devrez établir une correspondance avec les identifiants de la table *places*.

In [ ]:
# Your code here

# The set become a dict in which the key is the place
# and index the value
places = { place:idx for idx, place in enumerate(places) }

# A list of authors
authors = list()

for author in root.iter('author'):
    id_author = author.get('id_author')
    ref_birthplace = author.find('birth').get('place')
    ref_deathplace = author.find('death').get('place')
    lastname = author.find('lastname').text
    firstname = author.find('firstname').text
    birth = author.find('birth').text
    death = author.find('death').text
    authors.append(
        (id_author, places[ref_birthplace], places[ref_deathplace],
         lastname, firstname, birth, death)
    )

sql = "INSERT INTO authors VALUES (?, ?, ?, ?, ?, ?, ?)"
c.executemany(sql, authors)

Exécutez le code ci-dessous pour vérifier que vous disposez bien de quatre enregistrements :

In [ ]:
# Print all the records in the table 'authors'
for row in c.execute("SELECT * FROM authors"):
    print(row)

### Remplir la table *books*

Et pour la table *books* !

In [ ]:
# Your code here

# A list of books
books = list()

# A counter to keep track of when id_book is incremented
i = 0

for book in root.iter('book'):
    ref_author = book.get('ref_author')
    lang = book.get('{http://www.w3.org/XML/1998/namespace}lang')
    title = book.find('title').text
    original_title = book.find('title').get('original_title')
    original_title_lang = book.find('title').get('original_title_lang')
    pub_year = book.find('pub_year').text
    summary = book.find('summary').text
    grade = book.find('grade').text
    books.append(
        (i, ref_author, lang, title, original_title,
         original_title_lang, pub_year, summary, grade)
    )
    i += 1

sql = "INSERT INTO books VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"
c.executemany(sql, books)

Encore une fois, si tout s’est bien passé, le code ci-dessous devrait afficher la liste des livres :

In [ ]:
for row in c.execute("SELECT * FROM books"):
    print(row)

### Sauvegarder les transactions

En dernier lieu, n’oubliez pas d’enregistrer toutes vos transactions et de fermer la connexion à la base de données !

In [ ]:
# Your code here
connection.commit()
connection.close()

## Extraire des données

Tout est place, nous allons maintenant essayer d’extraire quelques informations de la base de données. Comme la connexion à été fermée précédemment, il faudra avant tout la rouvrir et instancier un nouvel objet `Cursor`.

In [ ]:
# Your code here
connection = sqlite3.connect('../data/library.db')
c = connection.cursor()

**Q1 :** Combien de livres existe-t-il dans la base ?

In [ ]:
# Your code here
sql = "SELECT COUNT(id_book) FROM books"
nb_books = c.execute(sql).fetchone()

**Q2 :** Quelle est la date de naissance de Flaubert ?

In [ ]:
# Your code here
sql = "SELECT birth FROM authors WHERE lastname = 'Flaubert'"
birth = c.execute(sql).fetchone()

**Q3 :** Listez, par ordre alphabétique, les titres des livres et leurs auteurs

In [ ]:
# Your code here
sql = """SELECT title, firstname, lastname
FROM books
INNER JOIN authors ON id_author = ref_author
ORDER BY title ASC"""
for row in c.execute(sql):
    print(f"{row[0]} de {row[1]} {row[2]}")